In [1]:
import json
import requests
import re
from IPython import display
from datetime import datetime
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import pandas as pd
import openpyxl

In [2]:
FIREBASE_PRIVATE_KEY = r'C:\Users\johnp\Documents\ESportsAnalytics\esportsanalytics-3b04cf0c1d47.json'
BASE_URL = 'https://api.opendota.com/api/'
TEAMS = 'teams'
HEROES = 'heroes'
PROPLAYERS = 'proPlayers'
PROMATCHES = 'proMatches'
MATCHES = 'matches'
PLAYERS = 'players'
HEROES_STATS = 'heroStats'


In [3]:
# Connect to Firebase and return a database instance
def connecttofirebase(PATH):
    
    # Check if the app is already initialised, if not then connect
    if not firebase_admin._apps:
        cred = credentials.Certificate(FIREBASE_PRIVATE_KEY)
        firebase_admin.initialize_app(cred)
    
    # Get an instance of the project database
    db = firestore.client()
    
    return db

In [4]:
db = connecttofirebase(FIREBASE_PRIVATE_KEY)

In [5]:
db.project

'esportsanalytics'

# Data Extraction

In [6]:
def get_all_teams_from_firebase():
    
     # get the data from Firebase
    doc_ref = db.collection(u'dota').document(u'all_teams')
    doc = doc_ref.get()
    
    # get the upload time from Firebase
    datetime_upload_str = doc.to_dict()['date_time_upload']
    datetime_upload = datetime.strptime(datetime_upload_str, "%m/%d/%Y, %H:%M:%S")

    # subtracting two datetime objects result in a datetime.timedelta object
    data_age_timedelta = datetime.now() - datetime_upload
    data_age_in_days = data_age_timedelta.total_seconds() / 86400
    
    print('Data is:', data_age_in_days, 'old')
    
    # Get the data from Firebase
    team_data_json = doc.to_dict()['all_teams_info']
    
    return team_data_json

In [7]:
def get_all_teams_from_opendota():
    
    response = requests.get(BASE_URL + TEAMS)

    try:
        team_data_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
    
    else:
        return team_data_json
    
    return 0

In [8]:
def update_all_teams_in_firebase():
    
    response = requests.get(BASE_URL + TEAMS)

    try:
        team_data_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)

    else:
        # upload the team_data_json in Firebase if there are no errors
        data = {
            u'date_time_upload' : datetime.now().strftime("%m/%d/%Y, %H:%M:%S"),
            u'all_teams_info' : team_data_json
        }

        db.collection(u'dota').document(u'all_teams').set(data)
        print("All Teams data updated in Firebase")

In [9]:
# Save the JSON as a file in the working directory
def save_all_teams_data_to_file(all_teams_data_json):
    
    with open('all_teams_data.json', 'w', encoding='utf-8') as f:
        json.dump(all_teams_data_json, f, ensure_ascii=False, indent=4)

In [10]:
def get_all_heroes_from_firebase():
    
    # get the data from Firebase
    doc_ref = db.collection(u'dota').document(u'all_heroes')
    doc = doc_ref.get()
    
    # get the upload time from Firebase
    datetime_upload_str = doc.to_dict()['date_time_upload']
    datetime_upload = datetime.strptime(datetime_upload_str, "%m/%d/%Y, %H:%M:%S")

    # subtracting two datetime objects result in a datetime.timedelta object
    data_age_timedelta = datetime.now() - datetime_upload
    data_age_in_days = data_age_timedelta.total_seconds() / 86400
    
    print('Data is:', data_age_in_days, 'old')
    
    # Get the data from Firebase
    heroes_data_json = doc.to_dict()['all_heroes_info']
    
    return heroes_data_json

In [11]:
def get_all_heroes_from_opendota():
    response = requests.get(BASE_URL + HEROES)

    try:
        heroes_data_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
    
    else:
        return heroes_data_json
    
    return 0

In [12]:
def update_all_heroes_in_firebase():
    
    response = requests.get(BASE_URL + HEROES)

    try:
        heroes_data_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)

    else:
        # upload the team_data_json in Firebase if there are no errors
        data = {
            u'date_time_upload' : datetime.now().strftime("%m/%d/%Y, %H:%M:%S"),
            u'all_teams_info' : heroes_data_json
        }

        db.collection(u'dota').document(u'all_heroes').set(data)
        print("All Heroes data updated in Firebase")

In [13]:
# Save the JSON as a file in the working directory
def save_all_heroes_data_to_file(all_heroes_data_json):
    
    with open('all_heroes_data.json', 'w', encoding='utf-8') as f:
        json.dump(all_heroes_data_json, f, ensure_ascii=False, indent=4)

In [14]:
def get_all_pro_players_in_opendota():
    
    response = requests.get(BASE_URL + PROPLAYERS)

    try:
        all_pro_players_data_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
        
    else:
        return all_pro_players_data_json
    
    return 0

In [15]:
def get_all_pro_players_in_firebase():
    all_pro_players_data_json = []
    
    firebase_doc_list = [
        u'all_pro_players_1',
        u'all_pro_players_2',
        u'all_pro_players_3'
    ]
    
    for index, item in enumerate(firebase_doc_list):
        
        doc_ref = db.collection(u'dota').document(item)
        doc = doc_ref.get()
        
        # Get the data from Firebase
        all_pro_players_data_json += doc.to_dict()['all_pro_players_info_'+str(index+1)]
    
    return all_pro_players_data_json

In [16]:
def update_all_pro_players_in_firebase():

    # Do these if data doesn't exist yet or is not up to date
    response = requests.get(BASE_URL + PROPLAYERS)

    try:
        pro_players_data_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)

    else:

        pro_players_split = [
            {u'all_pro_players_info_1' : pro_players_data_json[0:1000], u'date_time_upload' : datetime.now().strftime("%m/%d/%Y, %H:%M:%S"),},
            {u'all_pro_players_info_2' : pro_players_data_json[1000:2000], u'date_time_upload' : datetime.now().strftime("%m/%d/%Y, %H:%M:%S"),},
            {u'all_pro_players_info_3' : pro_players_data_json[2000:], u'date_time_upload' : datetime.now().strftime("%m/%d/%Y, %H:%M:%S"),},

        ]

        for index, data in enumerate(pro_players_split):
            pro_players_ref = db.collection(u'dota').document(u'all_pro_players_'+str(index+1)).set(data)

        print("All Pro Players data updated in Firebase")
    

In [17]:
# Save the JSON as a file in the working directory
def save_all_pro_players_data_to_file(all_pro_players_data_json):
    
    with open('all_pro_players_data.json', 'w', encoding='utf-8') as f:
        json.dump(all_pro_players_data_json, f, ensure_ascii=False, indent=4)

In [18]:
def get_opendota_api_info():
    
    response = requests.get(BASE_URL)

    try:
        opendota_api_data_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
        
    else:
        return opendota_api_data_json
    
    return 0

In [19]:
def get_match_info(match_id):
    response = requests.get(BASE_URL + MATCHES + '/' + str(match_id))

    try:
        match_info_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
        
    else:
        return match_info_json
    
    return 0

In [20]:
def get_all_heroes_stats():
    response = requests.get(BASE_URL + HEROES_STATS)

    try:
        all_heroes_stats_data_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
        
    else:
        return all_heroes_stats_data_json
    
    return 0

# Data Search

In [21]:
def search_regex(search_string, name_from_list):
    try:
        x = re.search(search_string, name_from_list, re.IGNORECASE).group()
    except Exception:
        return False
    return x

In [22]:
def search_item(item_to_search, item_key, data_source):
    item_filter =  filter(lambda y: search_regex(item_to_search, y[item_key]), data_source)
    return [item_info for item_info in item_filter]

In [23]:
all_teams = get_all_teams_from_opendota()

In [24]:
x = search_item('fnatic', 'name', all_teams)
x[0]

{'team_id': 350190,
 'rating': 1403.13,
 'wins': 1153,
 'losses': 845,
 'last_match_time': 1661607736,
 'name': 'Fnatic',
 'tag': 'Fnatic',
 'logo_url': 'https://steamcdn-a.akamaihd.net/apps/dota2/images/team_logos/350190.png'}

In [25]:
def get_team_matches(team_id):
    
    response = requests.get(BASE_URL + TEAMS + '/' + str(team_id) + '/' + MATCHES)
    print(response)

    try:
        team_data_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
    
    else:
        return team_data_json
    
    return 0

In [26]:
team_fnatic_matches = get_team_matches(350190)

<Response [200]>


In [27]:
len(team_fnatic_matches)

2081

In [28]:
team_og_matches = get_team_matches(2586976)

<Response [200]>


In [29]:
len(team_og_matches)

1333

with open('team_og_matches.json', 'w', encoding='utf-8') as f:
    json.dump(team_og_matches, f, ensure_ascii=False, indent=4)

In [30]:
all_heroes = get_all_heroes_from_opendota()

In [31]:
all_pro_players = get_all_pro_players_in_opendota()

In [32]:
def match_outcome(radiant_win, radiant):
    if ((radiant_win == False and radiant == False) or (radiant_win == True and radiant == True)):
        return True
    
    elif ((radiant_win == True and radiant == False) or (radiant_win == False and radiant == True)):
        return False

In [33]:
# Convert the team matches json to a dataframe
def team_matches_df(team_matches_json):
    
    all_matches_df = pd.DataFrame(data = [(datetime.fromtimestamp(item['start_time']).strftime('%Y-%m-%d %H:%M:%S'), 
                                                       item['opposing_team_name'],
                                                       item['opposing_team_id'], 
                                                       item['league_name'],
                                                       item['leagueid'],
                                                       item['cluster'], 
                                                       item['radiant_win'],
                                                       item['radiant'],
                                                       item['duration'], 
                                                       item['match_id'],
                                                       match_outcome(item['radiant_win'], item['radiant'])) for item in team_matches_json], 
                      columns = ['start_date', 'opposing_team_name', 'opposing_team_id', 'league_name', 'leagueid', 'cluster', 'radiant_win', 'radiant', 'duration', 'match_id', 'match_outcome'])
    
    return all_matches_df


In [34]:
team_og_matches_df = team_matches_df(team_og_matches)

In [35]:
team_og_matches_df.shape

(1333, 11)

In [36]:
team_og_matches_df.tail()

,start_date,opposing_team_name,opposing_team_id,league_name,leagueid,cluster,radiant_win,radiant,duration,match_id,match_outcome
1328,2015-11-04 04:50:57,MONKEY FREEDOM FIGHTERS,758797,ASUS ROG DreamLeague Season 4,3865,133,True,True,1423,1912135868,True
1329,2015-11-03 08:09:58,Mamas-Boys,2276247,ASUS ROG DreamLeague Season 4,3865,138,True,True,1092,1910208777,True
1330,2015-11-03 07:00:03,Mamas-Boys,2276247,ASUS ROG DreamLeague Season 4,3865,133,True,True,2464,1910121231,True
1331,2015-11-03 03:56:42,STARK.gg,1804536,StarLadder | i-League,3902,138,True,True,1059,1909825881,True
1332,2015-11-03 03:04:50,STARK.gg,1804536,StarLadder | i-League,3902,133,True,True,1517,1909728487,True


In [37]:
team_fnatic_matches = team_matches_df(team_fnatic_matches)

In [38]:
team_fnatic_matches.tail()

,start_date,opposing_team_name,opposing_team_id,league_name,leagueid,cluster,radiant_win,radiant,duration,match_id,match_outcome
2076,2013-02-17 04:59:50,Team Empire,46,RaidCall Dota 2 League Season 2,22,181,True,False,2689,127380209,False
2077,2013-02-17 04:19:54,Team Empire,46,RaidCall Dota 2 League Season 2,22,181,True,True,898,127335245,True
2078,2013-02-15 08:25:19,Natus Vincere,36,The Defense 3,21,191,False,False,2214,125548077,True
2079,2013-02-15 07:41:12,Natus Vincere,36,The Defense 3,21,191,False,False,1431,125525847,True
2080,2013-02-15 06:41:52,Natus Vincere,36,The Defense 3,21,191,False,True,2611,125486981,False


In [39]:
og_v_fnatic_match_info = get_match_info(6720651873)

In [40]:
len(og_v_fnatic_match_info)

46

In [41]:
for key in og_v_fnatic_match_info:
    print(key)

match_id
barracks_status_dire
barracks_status_radiant
chat
cluster
cosmetics
dire_score
dire_team_id
draft_timings
duration
engine
first_blood_time
game_mode
human_players
leagueid
lobby_type
match_seq_num
negative_votes
objectives
picks_bans
positive_votes
radiant_gold_adv
radiant_score
radiant_team_id
radiant_win
radiant_xp_adv
skill
start_time
teamfights
tower_status_dire
tower_status_radiant
version
replay_salt
series_id
series_type
league
radiant_team
dire_team
players
patch
region
all_word_counts
my_word_counts
throw
loss
replay_url


In [42]:
players = og_v_fnatic_match_info['players']

In [43]:
[p_keys for p_keys in players[0]]

['match_id',
 'player_slot',
 'ability_targets',
 'ability_upgrades_arr',
 'ability_uses',
 'account_id',
 'actions',
 'additional_units',
 'assists',
 'backpack_0',
 'backpack_1',
 'backpack_2',
 'backpack_3',
 'buyback_log',
 'camps_stacked',
 'connection_log',
 'creeps_stacked',
 'damage',
 'damage_inflictor',
 'damage_inflictor_received',
 'damage_taken',
 'damage_targets',
 'deaths',
 'denies',
 'dn_t',
 'firstblood_claimed',
 'gold',
 'gold_per_min',
 'gold_reasons',
 'gold_spent',
 'gold_t',
 'hero_damage',
 'hero_healing',
 'hero_hits',
 'hero_id',
 'item_0',
 'item_1',
 'item_2',
 'item_3',
 'item_4',
 'item_5',
 'item_neutral',
 'item_uses',
 'kill_streaks',
 'killed',
 'killed_by',
 'kills',
 'kills_log',
 'lane_pos',
 'last_hits',
 'leaver_status',
 'level',
 'lh_t',
 'life_state',
 'max_hero_hit',
 'multi_kills',
 'net_worth',
 'obs',
 'obs_left_log',
 'obs_log',
 'obs_placed',
 'party_id',
 'party_size',
 'performance_others',
 'permanent_buffs',
 'pings',
 'pred_vict',
 

In [44]:
for i in range(5):
    
    print(players[i]['name'], players[i]['lane_role'], players[i]['lane'], players[i]['account_id'])

AMMAR_THE_F 3 3 183719386
bzm 2 2 93618577
Yuragi 1 1 167976729
Chu 1 1 117483894
Taiga 3 3 401792574


In [45]:
def get_team_players(team_id):
    response = requests.get(BASE_URL + TEAMS + '/' + str(team_id) + '/' + PLAYERS)
    print(response)

    try:
        team_data_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
    
    else:
        return team_data_json
    
    return 0

In [46]:
team_og_players = get_team_players(2586976)

<Response [200]>


In [47]:
team_og_players[0]

{'account_id': 19672354,
 'name': 'N0tail',
 'games_played': 1150,
 'wins': 709,
 'is_current_team_member': False}

In [48]:
player = search_item('ana', 'name', all_pro_players)

In [49]:
[item for item in all_heroes if item['id'] == 74]

[{'id': 74,
  'name': 'npc_dota_hero_invoker',
  'localized_name': 'Invoker',
  'primary_attr': 'int',
  'attack_type': 'Ranged',
  'roles': ['Carry', 'Nuker', 'Disabler', 'Escape', 'Pusher'],
  'legs': 2}]

In [50]:
ana_account_id = 311360822

In [51]:
[item for item in all_pro_players if item['account_id'] == 70388657]

[{'account_id': 70388657,
  'steamid': '76561198030654385',
  'avatar': 'https://avatars.akamai.steamstatic.com/a1e7773eb1140703253265df18c608ec3e91531e.jpg',
  'avatarmedium': 'https://avatars.akamai.steamstatic.com/a1e7773eb1140703253265df18c608ec3e91531e_medium.jpg',
  'avatarfull': 'https://avatars.akamai.steamstatic.com/a1e7773eb1140703253265df18c608ec3e91531e_full.jpg',
  'profileurl': 'https://steamcommunity.com/id/DendiQ/',
  'personaname': 'Dendalf',
  'last_login': None,
  'full_history_time': '2022-08-26T04:02:26.789Z',
  'cheese': 0,
  'fh_unavailable': False,
  'loccountrycode': 'UA',
  'last_match_time': '2022-08-28T00:12:37.000Z',
  'plus': True,
  'name': 'Dendi',
  'country_code': '',
  'fantasy_role': 4,
  'team_id': 8359896,
  'team_name': 'B8',
  'team_tag': 'B8',
  'is_locked': True,
  'is_pro': True,
  'locked_until': None}]

In [52]:
all_heroes_stats = get_all_heroes_stats()

In [53]:
all_heroes_stats[0]

{'id': 1,
 'name': 'npc_dota_hero_antimage',
 'localized_name': 'Anti-Mage',
 'primary_attr': 'agi',
 'attack_type': 'Melee',
 'roles': ['Carry', 'Escape', 'Nuker'],
 'img': '/apps/dota2/images/dota_react/heroes/antimage.png?',
 'icon': '/apps/dota2/images/dota_react/heroes/icons/antimage.png?',
 'base_health': 200,
 'base_health_regen': 0.25,
 'base_mana': 75,
 'base_mana_regen': 0,
 'base_armor': 0,
 'base_mr': 25,
 'base_attack_min': 29,
 'base_attack_max': 33,
 'base_str': 21,
 'base_agi': 24,
 'base_int': 12,
 'str_gain': 1.6,
 'agi_gain': 2.8,
 'int_gain': 1.8,
 'attack_range': 150,
 'projectile_speed': 0,
 'attack_rate': 1.4,
 'move_speed': 310,
 'turn_rate': None,
 'cm_enabled': True,
 'legs': 2,
 'hero_id': 1,
 'turbo_picks': 404987,
 'turbo_wins': 206578,
 'pro_ban': 104,
 'pro_win': 21,
 'pro_pick': 44,
 '1_pick': 21033,
 '1_win': 10218,
 '2_pick': 41847,
 '2_win': 20138,
 '3_pick': 41324,
 '3_win': 19869,
 '4_pick': 31777,
 '4_win': 14975,
 '5_pick': 16904,
 '5_win': 7973,


# What makes a team win matches?

-players? Team composition

-draft pick? Hero composition

-items? Item-Hero compatibility

-time? Match duration

# What makes a specific team win?

Since you can't include team names in the dataset, teams should be represented numerically with information that excludes historical match performance data (because new data pre-hero-draft won't include match data of course)

The only way I can think of is to use that specific match's player's historical performance data leading up to that match as a representation of the teams e.g. 

Team OG Matches from 2015:

	    start_date	        opposing_team_name	    opposing_team_id	league_name	leagueid	        cluster	radiant_win	radiant	duration	match_id	match_outcome
        
1328	2015-11-04 04:50:57	MONKEY FREEDOM FIGHTERS	758797	            ASUS ROG DreamLeague Season 4	3865	133	True	True	1423	1912135868	True

1329	2015-11-03 08:09:58	Mamas-Boys	            2276247	            ASUS ROG DreamLeague Season 4	3865	138	True	True	1092	1910208777	True

For match id 1912135868, include all 5 player performance data from the lead up to that 2015 match and don't include future data beyond 2015 even if we have it!

# Figure out which metadata to use to quantify player performance

e.g. lane efficiency, MMR, years experience, team cohesion

# Figure out which metadata to use to quantify team performance

e.g. team fight success rate, team brand goodwill? Team Rank

In [54]:
opendota_api_info = get_opendota_api_info()